In [1]:
import datetime
import json
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm as tqdm_notebook

In [2]:
def analyse(symbol : str) -> bool:
    today : str = str(datetime.datetime.now().date())
    one_year_ago = str((datetime.datetime.now() - datetime.timedelta(days=365)).date())
    history_data_frame = pd.read_csv(f"./data/history/all/{symbol}.csv")
    history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
    one_year_data_frame = history_data_frame.loc[(history_data_frame['date'] >= one_year_ago) & (history_data_frame['date'] <= today) & (history_data_frame['volume'] > 0)]
    print(symbol, one_year_ago, today, len(one_year_data_frame))
    if len(one_year_data_frame) < 200:
        return False
    # Past
    history_data_frame["price_three_days_ago"] = history_data_frame["close"].shift(3)
    history_data_frame["price_three_days_ago_difference"] = round(history_data_frame["close"] - history_data_frame["price_three_days_ago"], 2)
    history_data_frame["price_three_days_ago_return"] = round(history_data_frame["price_three_days_ago_difference"] / history_data_frame["price_three_days_ago"], 2)
    history_data_frame["price_two_days_ago"] = history_data_frame["close"].shift(2)
    history_data_frame["price_yesterday"] = history_data_frame["close"].shift(1)
    # Future
    history_data_frame["price_tomorrow"] = history_data_frame["close"].shift(-1)
    history_data_frame["price_tomorrow_difference"] = round(history_data_frame["price_tomorrow"] - history_data_frame["close"], 2)
    history_data_frame["price_tomorrow_return"] = round(history_data_frame["price_tomorrow_difference"] / history_data_frame["close"], 2)
    # Direction
    history_data_frame["direction"] = [1 if history_data_frame.loc[ei, "price_tomorrow_difference"] > 0 else -1 for ei in history_data_frame.index]
    # Moving Average
    history_data_frame["MA5"] = round(history_data_frame["close"].rolling(5).mean(), 2)
    history_data_frame["MA10"] = round(history_data_frame["close"].rolling(10).mean(), 2)
    history_data_frame["MA20"] = round(history_data_frame["close"].rolling(20).mean(), 2)
    history_data_frame["MA50"] = round(history_data_frame["close"].rolling(50).mean(), 2)
    history_data_frame["MA100"] = round(history_data_frame["close"].rolling(100).mean(), 2)
    history_data_frame["MA200"] = round(history_data_frame["close"].rolling(200).mean(), 2)
    # Indicator
    history_data_frame["MA50-1"] = history_data_frame["MA50"].shift(1)
    history_data_frame["MA200-1"] = history_data_frame["MA200"].shift(1)
    history_data_frame["golden-cross-1"] = history_data_frame["MA50-1"] <= history_data_frame["MA200-1"]
    history_data_frame["golden-cross-0"] = history_data_frame["MA50"] >= history_data_frame["MA200"]
    history_data_frame["golden-cross"] = history_data_frame["golden-cross-1"] & history_data_frame["golden-cross-0"]
    # Save to File
    plt.plot(history_data_frame[['close', 'MA50', 'MA200']].tail(200))
    plt.legend(['close', 'MA50', 'MA200'], loc='upper left')
    first : str = symbol[0].lower()
    plt.savefig(f"./images/{first}/{symbol}.png")
    plt.clf()
    history_data_frame.to_csv(f"./data/analyse/{first}/{symbol}.csv",index=False)
    if True in list(history_data_frame["golden-cross"].tail(5)):
        print(f"There is golden cross in {symbol}")
    return True in list(history_data_frame["golden-cross"].tail(5))

In [3]:
vn30_file = open("./vn30.json", 'r')
vn30 : list[str] = json.load(vn30_file)

golden_cross_vn30 = []

for symbol in tqdm_notebook(vn30):
    golden_cross = analyse(symbol)
    if golden_cross:
        golden_cross_vn30.append(symbol)

golden_cross_vn30

  0%|          | 0/30 [00:00<?, ?it/s]

ACB 2022-11-25 2023-11-25 250
BCM 2022-11-25 2023-11-25 250
BID 2022-11-25 2023-11-25 250
BVH 2022-11-25 2023-11-25 250
CTG 2022-11-25 2023-11-25 250
FPT 2022-11-25 2023-11-25 250
GAS 2022-11-25 2023-11-25 250
GVR 2022-11-25 2023-11-25 250
HDB 2022-11-25 2023-11-25 250
HPG 2022-11-25 2023-11-25 250
MBB 2022-11-25 2023-11-25 250
MSN 2022-11-25 2023-11-25 250
MWG 2022-11-25 2023-11-25 250
PLX 2022-11-25 2023-11-25 250
POW 2022-11-25 2023-11-25 250
SAB 2022-11-25 2023-11-25 250
SHB 2022-11-25 2023-11-25 250
SSB 2022-11-25 2023-11-25 250
SSI 2022-11-25 2023-11-25 250
STB 2022-11-25 2023-11-25 250
TCB 2022-11-25 2023-11-25 250
TPB 2022-11-25 2023-11-25 250
VCB 2022-11-25 2023-11-25 250
VHM 2022-11-25 2023-11-25 250
VIB 2022-11-25 2023-11-25 250
VIC 2022-11-25 2023-11-25 250
VJC 2022-11-25 2023-11-25 250
VNM 2022-11-25 2023-11-25 250
VPB 2022-11-25 2023-11-25 250
VRE 2022-11-25 2023-11-25 250


[]

<Figure size 640x480 with 0 Axes>

In [4]:
companies_data_frame = pd.read_csv("./data/companies.csv")
vnindex : list[str] = list(companies_data_frame['stock_code'])
vnindex = list(filter(lambda s: len(s) == 3, vnindex))
vnindex

['A32',
 'AAA',
 'AAM',
 'AAS',
 'AAT',
 'AAV',
 'ABB',
 'ABC',
 'ABI',
 'ABR',
 'ABS',
 'ABT',
 'ABW',
 'ACB',
 'ACC',
 'ACE',
 'ACG',
 'ACL',
 'ACM',
 'ACS',
 'ACV',
 'ADC',
 'ADG',
 'ADP',
 'ADS',
 'AFX',
 'AG1',
 'AGE',
 'AGF',
 'AGG',
 'AGM',
 'AGP',
 'AGR',
 'AGX',
 'AIC',
 'ALT',
 'ALV',
 'AMC',
 'AMD',
 'AME',
 'AMP',
 'AMS',
 'AMV',
 'ANT',
 'ANV',
 'APC',
 'APF',
 'APG',
 'APH',
 'API',
 'APL',
 'APP',
 'APS',
 'APT',
 'ARM',
 'ART',
 'ASA',
 'ASG',
 'ASM',
 'ASP',
 'AST',
 'ATA',
 'ATB',
 'ATG',
 'ATS',
 'AUM',
 'AVC',
 'AVF',
 'B82',
 'BAB',
 'BAF',
 'BAL',
 'BAX',
 'BBC',
 'BBH',
 'BBM',
 'BBS',
 'BBT',
 'BCA',
 'BCB',
 'BCC',
 'BCE',
 'BCF',
 'BCG',
 'BCM',
 'BCP',
 'BCV',
 'BDB',
 'BDG',
 'BDT',
 'BDW',
 'BED',
 'BEL',
 'BFC',
 'BGW',
 'BHA',
 'BHC',
 'BHG',
 'BHI',
 'BHK',
 'BHN',
 'BHP',
 'BHT',
 'BIC',
 'BID',
 'BIG',
 'BII',
 'BIO',
 'BKC',
 'BKG',
 'BKH',
 'BLF',
 'BLI',
 'BLN',
 'BLT',
 'BLW',
 'BMC',
 'BMD',
 'BMF',
 'BMG',
 'BMI',
 'BMJ',
 'BMN',
 'BMP',
 'BMS',


In [5]:
golden_cross_vnindex = []

for symbol in tqdm_notebook(vnindex):
    golden_cross = analyse(symbol)
    if golden_cross:
        golden_cross_vnindex.append(symbol)

golden_cross_vnindex

  0%|          | 0/1615 [00:00<?, ?it/s]

A32 2022-11-25 2023-11-25 97
AAA 2022-11-25 2023-11-25 250
AAM 2022-11-25 2023-11-25 249
AAS 2022-11-25 2023-11-25 250
AAT 2022-11-25 2023-11-25 250
AAV 2022-11-25 2023-11-25 250
ABB 2022-11-25 2023-11-25 250
ABC 2022-11-25 2023-11-25 177
ABI 2022-11-25 2023-11-25 249
ABR 2022-11-25 2023-11-25 203
ABS 2022-11-25 2023-11-25 250
ABT 2022-11-25 2023-11-25 200
ABW 2022-11-25 2023-11-25 127
ACB 2022-11-25 2023-11-25 250
ACC 2022-11-25 2023-11-25 250
ACE 2022-11-25 2023-11-25 166
ACG 2022-11-25 2023-11-25 250
ACL 2022-11-25 2023-11-25 250
ACM 2022-11-25 2023-11-25 51
ACS 2022-11-25 2023-11-25 56
ACV 2022-11-25 2023-11-25 250
ADC 2022-11-25 2023-11-25 168
ADG 2022-11-25 2023-11-25 242
ADP 2022-11-25 2023-11-25 178
ADS 2022-11-25 2023-11-25 250
AFX 2022-11-25 2023-11-25 250
AG1 2022-11-25 2023-11-25 162
AGE 2022-11-25 2023-11-25 19
AGF 2022-11-25 2023-11-25 51
AGG 2022-11-25 2023-11-25 250
AGM 2022-11-25 2023-11-25 200
AGP 2022-11-25 2023-11-25 241
AGR 2022-11-25 2023-11-25 250
AGX 2022-11-25 

['AIC', 'CLC', 'OPC', 'PSP']

<Figure size 640x480 with 0 Axes>

In [6]:
for symbol in golden_cross_vnindex:
    first : str = symbol[0].lower()
    print(f"./images/{first}/{symbol}.png")

./images/a/AIC.png
./images/c/CLC.png
./images/o/OPC.png
./images/p/PSP.png
